# Code Validation / Refactoring
## Non-functionalized versions for easier trouble shooting

In [ ]:
#no_test
#dependencies

#nlp packages
import spacy
from spacy.util import minibatch, compounding

#manipulation of tables/arrays
import pandas as pd
import numpy as np
import copy
import json

import difflib
import matplotlib.pyplot as plt
import matplotlib as mpl

#internal imports
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.model_performance_utils import *
from ssda_nlp.xml_parser import *
from ssda_nlp.unstructured2markup import *
from ssda_nlp.utility import *
from ssda_nlp.relationships import *
from ssda_nlp.full_volume import *

In [ ]:
import unicodedata

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

my_string = 'àéêöhello'
print(my_string)
s = strip_accents(my_string)
print(s)

my_string = "á, é, í, ó, ú, ü, ñ, ¿, ¡, ?, #,  , !, $, @, ."
print(my_string)
s = strip_accents(my_string)
print(s)

In [ ]:
#no_test

for person_idx in range(len(entry_people)):
    relationships = entry_people[person_idx].get('relationships')
    
    #are there any people with very similar names that still appear separately
    nameTemp = entry_people[person_idx].get('name')
    name_list.append(nameTemp)
    
    #1: Short name that could be a first name
    #Make a boolean list to flag which names may be first names (based on length)
    possibleFirstNames = []
    fullNames = []
    for name in name_list:
        if name in all_first_names:
            possibleFirstNames.append(name)
        else:
            fullNames.append(name)
    #Check to see if names appear within each other (i.e. is a person double counted)
    doubleCountedNames = []
    for idx in range(len(possibleFirstNames)):
        doubleCountedNames= doubleCountedNames + ([name for name in fullNames if possibleFirstNames[idx] in name])
    if not len(doubleCountedNames)==0:
        print("Possible double count (first name appears in a second instance (full name))")
        print(name_list)
        similarNames = 1
        
    #2: Two similarly-sized names, that could be variations (i.e. missing hypens or have #'s for unknown letters')
    # This is a bad check that needs to be replaced
    # Doesn't even check to see if composition is similar, only length...
    for name_idx in range(len(name_list)):
        # Why is it even passing here?
        if name_idx==0:
            pass
        else:
            for idx in range(len(name_list)):
                for idx2 in range(len(name_list)-idx-1):
                    idx2 = idx2+idx+1
                    if check_lengths(name_list, idx, idx2):
                        print("Similar Names: similar size")
                        print(name_list)
                        similarNames = 1 
                        
    #3: Strip spaces, ".", "#", convert accented characters to unaccented
    for name in name_list:
        # Strip any accents
        utf_name = strip_accents(name)
        # Remove bad characters
        condensed_name = utf_name.translate({ord(c): None for c in './\? !@#$'})
        # ^ How to check for similarity when # appears in one name but the actual letter does in the other?
        

In [14]:
#á, é, í, ó, ú, ü, ñ

name_list = ["Kai Malcolm", "Kai", "Kai Devon Malcolm", "Devon Malcolm", 
             "Dan Jenkins", "Daniel Jenkins", "Daniel Genkins", "Da##iel Genkin#", "D#l Ge#ins", 
            "Maria Andoval", "María Andoval", "Máríá ##o#al",
            "María Dolores", "María Dolores Sanchez", "Maria Sanchez"]

In [ ]:
#3: Strip spaces, ".", "#", convert accented characters to unaccented
for name in name_list:
    # Strip any accents
    utf_name = strip_accents(name)
    # Remove bad characters
    condensed_name = utf_name.translate({ord(c): None for c in './\? !@#$'})
    # ^ How to check for similarity when # appears in one name but the actual letter does in the other?

In [13]:
from difflib import SequenceMatcher
s_1 = 'Mohan Mehta'
s_2 = 'Mohan Mehte'
print(SequenceMatcher(a=s_1,b=s_2).ratio())

0.9090909090909091


In [19]:
from difflib import SequenceMatcher
s_1 = 'Maria Dolores Sanchez'
s_2 = 'Juan Dolores Sanchez'
print(SequenceMatcher(a=s_1,b=s_2).ratio())

0.8292682926829268


In [18]:
for name_idx in range(len(name_list)):
    try:
        name1 = name_list[name_idx]
        name2 = name_list[name_idx+1]
        # Strip any accents, Remove bad characters
        n1 = strip_accents(name1).translate({ord(c): None for c in './\? !@#$'})
        n2 = strip_accents(name2).translate({ord(c): None for c in './\? !@#$'})
        
        print(f"{name_list[name_idx]}, {name_list[name_idx+1]}, {SequenceMatcher(a=n1,b=n2).ratio():0.3f}")
        print("---------------------")
    except:
        pass

Kai Malcolm, Kai, 0.462
---------------------
Kai, Kai Devon Malcolm, 0.333
---------------------
Kai Devon Malcolm, Devon Malcolm, 0.889
---------------------
Devon Malcolm, Dan Jenkins, 0.273
---------------------
Dan Jenkins, Daniel Jenkins, 0.870
---------------------
Daniel Jenkins, Daniel Genkins, 0.923
---------------------
Daniel Genkins, Da##iel Genkin#, 0.917
---------------------
Da##iel Genkin#, D#l Ge#ins, 0.667
---------------------
D#l Ge#ins, Maria Andoval, 0.105
---------------------
Maria Andoval, María Andoval, 1.000
---------------------
María Andoval, Máríá ##o#al, 0.800
---------------------
Máríá ##o#al, María Dolores, 0.700
---------------------
María Dolores, María Dolores Sanchez, 0.774
---------------------
María Dolores Sanchez, Maria Sanchez, 0.774
---------------------


In [ ]:
entry_ex1 = ['María Dolores Sanchez', 'Don Miguel o’Reilly', 'Juan', 'Deliby', 'Don Francisco Sanchez', 'Maria Dolores', 'José Rivas', 'María de la Luz Blanco']
entry_ex2 = ['Antonio Guillo', 'Don Miguel o’Reilly', 'Isaac Guillo', 'Sara Ca mel', 'Juan Antonio', 'Antonio Pellice', 'Susana Pellicer']
entry_ex3 = ['María Juana Francisca Fish', 'Don Miguel o’Reilly', 'Don Jeremías Fish', 'Eva Fish', 'Maria Juana Francisca', 'Lorenzo Capó', 'Juana Joaquina Gonzalez']
#'Teniente de Cura Beneficiado'


In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()